In [3]:
import pandas as pd
import numpy as np

* T: target cells （CD4$^+$ T cells)
* I: productively infected CD4$^+$ T cells,
* V: free virus

\begin{align}
\frac{dT}{dt} &= \lambda -dT -\beta(t)TV, \quad T(0) =T_0\\
\frac{dI}{dt} &= \beta(t) TV - \delta I,\quad  I(0)= I_0 \\
\frac{dV}{dt} &= pI - cV, \quad V(0) = V_0
\end{align}
where 
\begin{align}
\beta(t) = 
\begin{cases}
\beta_0, \quad t\leq \tau \\
\beta_{\infty} + (\beta_0 - \beta_{\infty}) e^{-k(t-\tau)}, \quad t>\tau
\end{cases}
\end{align}

Determine the parameters $(\beta_0,\beta_{\infty},\lambda, d, \delta, p, c, k, \tau)$ so that it can fit the real data


You can also use odeit() https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html


Forward Euler 

\begin{align}
T(t+h) - T(t) &= \lambda - dT(t) - \beta(t) T(t)V(t)\\
I(t+h) - I(t) &= \beta(t) T(t)V(t) - \delta I(t) \\
V(t+h) - V(t) &= pI(t) - cV(t)
\end{align}

Backward Euler

\begin{align}
T(t+h) - T(t) &= \lambda - dT(t+h) - \beta(t+h) T(t+h)V(t+h)\\
I(t+h) - I(t) &= \beta(t+h) T(t+h) V(t+h) - \delta I(t+h) \\
V(t+h) - V(t) &= pI(t+h) - cV(t+h)
\end{align}

<!-- Then we have 
\begin{align}
T(t+h) - T(t) &= 


\end{align} -->



In [7]:
data = pd.read_csv('table2.csv') 
data

,Patient,Days post Infection,vRNA copies/ml,IgM (O.D.),IgG (O.D.)
0,CHID46,13,10910,-,-
1,CHID46,16,108100,0.026,0.268
2,CHID46,20,800000,0.371,0.354
3,CHID46,23,922270,1.927,2.047
4,CHID46,27,340400,1.512,2.335
...,...,...,...,...,...
62,CHID08,19,5631550,0.014,0.175
63,CHID08,24,6486240,0.114,0.258
64,CHID08,26,2296060,0.795,0.585
65,CHID08,31,26311,0.997,1.456


In [12]:
T_0 = 10**4 
I_0 = 0
V_0 = 10**-3
c = 23 # day^-1  p.2
# lbda = d * T_0 
tau = 1 # try 1 day in increments



In [2]:
def beta(t, tau, k):
    if t <= tau:
        return beta_0
    else:
        return beta_infty + (beta_0-beta_infty)*exp(-k*(t-tau))

In [4]:
# T_end = 1 
# N = 5

# h = T/N
# t = 0

# old_T = T_0
# old_I = I_0
# old_V = V_0 


# while t < T_end:
#     # Forward Euler
#     new_T = old_T + d*T_0 - d*old_T - beta(t,tau,k)*old_T*old_V 
#     new_I = old_I + beta(t,tau,k)*old_T*old_v - delta*old_I
#     new_V = old_V + p*old_I - c*old_V
    
#     # Backward Euler
#     new_T = old_T + lbda - d*old_T - beta(t,tau,k)*old_T*old_V 
#     new_I = old_I + beta(t,tau,k)*old_T*old_V - delta*old_I
#     new_V = old_V + p*old_I - c*old_V    
    
    
#     old_T = new_T
#     old_I = new_I
#     old_V = new_V
    
#     t += h

In [ ]:
T_end = 1 
N = 5

h = T/N
t = 0

T = N * [T_0]
I = N * [I_0]
V = N * [V_0]

i = 0
while t < T_end:
    # Forward Euler
    T[i+1] = T[i] + d*T_0 - d*T[i] - beta(t,tau,k)*T[i]*V[i]
    I[i+1] = I[i] + beta(t,tau,k)*T[i]*V[i] - delta*I[i]
    V[i+1] = V[i] + p*I[i] - c*V[i]
    
    i += 1
    t += h

def J(beta_0,beta_infty, k, delta, p, d):
    return 1/N * sum(([np.log10(V[i]) -np.log10(V_true[i])**2 for i in range(N)])    